In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline
# %config IPCompleter.greedy=True

In [2]:
df = pd.read_csv('train-data.csv')

In [3]:
df.shape

(6019, 14)

In [4]:
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6019 entries, 0 to 6018
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         6019 non-null   int64  
 1   Name               6019 non-null   object 
 2   Location           6019 non-null   object 
 3   Year               6019 non-null   int64  
 4   Kilometers_Driven  6019 non-null   int64  
 5   Fuel_Type          6019 non-null   object 
 6   Transmission       6019 non-null   object 
 7   Owner_Type         6019 non-null   object 
 8   Mileage            6017 non-null   object 
 9   Engine             5983 non-null   object 
 10  Power              5983 non-null   object 
 11  Seats              5977 non-null   float64
 12  New_Price          824 non-null    object 
 13  Price              6019 non-null   float64
dtypes: float64(2), int64(3), object(9)
memory usage: 658.5+ KB


In [6]:
df.describe()

,Unnamed: 0,Year,Kilometers_Driven,Seats,Price
count,6019.000000,6019.000000,6.019000e+03,5977.000000,6019.000000
mean,3009.000000,2013.358199,5.873838e+04,5.278735,9.479468
std,1737.679967,3.269742,9.126884e+04,0.808840,11.187917
min,0.000000,1998.000000,1.710000e+02,0.000000,0.440000
25%,1504.500000,2011.000000,3.400000e+04,5.000000,3.500000
50%,3009.000000,2014.000000,5.300000e+04,5.000000,5.640000
75%,4513.500000,2016.000000,7.300000e+04,5.000000,9.950000
max,6018.000000,2019.000000,6.500000e+06,10.000000,160.000000


In [7]:
# Check for null values
df.isna().sum()

Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  42
New_Price            5195
Price                   0
dtype: int64

### Data cleaning

In [8]:
# Since New-Price which contains original price of the car have lot of null values we will drop that

df.drop('New_Price', axis=1, inplace=True)

In [9]:
df['Seats'].isna().sum() /  df.shape[0]

0.006977903306197043

In [10]:
# Features 'Mileage', 'Engine', 'Power', 'Seats' contains very less amount i.e.less than 1 percent
# It's better to drop them for ease of model building

df.dropna(inplace=True)

In [11]:
df.isna().sum()

Unnamed: 0           0
Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Price                0
dtype: int64

In [12]:
# Also column 'Unnamed: 0' does not represent anything other than serial number, we will also drop that

df.drop('Unnamed: 0', axis=1, inplace=True)

In [13]:
df.shape

(5975, 12)

In [14]:
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74


### Feature Engineering

<b> We will create a new feature of name 'Brand' and also a whole new dataset containing only some specific values from feature brand. Here, I used some domain knowledge and considered brands that have strong market capture in India </b> 

In [15]:
df['Brand'] = df['Name'].apply(lambda x: str(x).split(" ")[0])
df['Brand'].head()

0     Maruti
1    Hyundai
2      Honda
3     Maruti
4       Audi
Name: Brand, dtype: object

In [17]:
# Subsetting the data based on certain brands in Indian auto market

db1 = df[df['Brand']=='Honda']
db2 = df[df['Brand']=='Maruti'] 
db3 = df[df['Brand']=='Hyundai']
db4 = df[df['Brand']=='Toyota']
db5 = df[df['Brand']=='Mercedes-Benz']
db6 = df[df['Brand']=='Volkswagen']

In [18]:
# joining all these subsets to form a new dataset of df4, you can name whatever you like

df4 = pd.concat([db1,db2,db3,db4,db5,db6], axis=0)

In [19]:
df4.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Brand
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50,Honda
11,Honda City 1.5 V AT Sunroof,Kolkata,2012,60000,Petrol,Automatic,First,16.8 kmpl,1497 CC,116.3 bhp,5.0,4.49,Honda
16,Honda Amaze S i-Dtech,Kochi,2016,58950,Diesel,Manual,First,25.8 kmpl,1498 CC,98.6 bhp,5.0,5.40,Honda
25,Honda City Corporate Edition,Mumbai,2012,51920,Petrol,Manual,First,16.8 kmpl,1497 CC,116.3 bhp,5.0,4.25,Honda
28,Honda WRV i-VTEC VX,Kochi,2018,37430,Petrol,Manual,First,17.5 kmpl,1199 CC,88.7 bhp,5.0,9.90,Honda


In [20]:
df4.shape

(3940, 13)

In [21]:
# Once again we will check for null values to be completely sure
df4.isna().sum()

Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Price                0
Brand                0
dtype: int64

In [22]:
# We will drop the feature 'Name' since we do not it any more

df4.drop('Name', axis=1, inplace=True)

In [23]:
df4.head()

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Brand
2,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50,Honda
11,Kolkata,2012,60000,Petrol,Automatic,First,16.8 kmpl,1497 CC,116.3 bhp,5.0,4.49,Honda
16,Kochi,2016,58950,Diesel,Manual,First,25.8 kmpl,1498 CC,98.6 bhp,5.0,5.40,Honda
25,Mumbai,2012,51920,Petrol,Manual,First,16.8 kmpl,1497 CC,116.3 bhp,5.0,4.25,Honda
28,Kochi,2018,37430,Petrol,Manual,First,17.5 kmpl,1199 CC,88.7 bhp,5.0,9.90,Honda


In [24]:
df4.shape

(3940, 12)

<b> We have this feature 'Year' in year format which represents the year or edition of the model. We will create a new column 'Age' substracting the year from present year 2020 </b>

In [25]:
df4['Age'] = 2020 - df4['Year'] 
df4['Age'].head()

2     9
11    8
16    4
25    8
28    2
Name: Age, dtype: int64

In [26]:
# Drop the column 'Year' as we have a new column inplace of it

df4.drop('Year', axis=1, inplace=True)

In [27]:
df4.head()

,Location,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Brand,Age
2,Chennai,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50,Honda,9
11,Kolkata,60000,Petrol,Automatic,First,16.8 kmpl,1497 CC,116.3 bhp,5.0,4.49,Honda,8
16,Kochi,58950,Diesel,Manual,First,25.8 kmpl,1498 CC,98.6 bhp,5.0,5.40,Honda,4
25,Mumbai,51920,Petrol,Manual,First,16.8 kmpl,1497 CC,116.3 bhp,5.0,4.25,Honda,8
28,Kochi,37430,Petrol,Manual,First,17.5 kmpl,1199 CC,88.7 bhp,5.0,9.90,Honda,2


In [28]:
# Clean up the feature 'Engine' and convert it to integer type for further processing and model building
df4['Engine'] = df4['Engine'].apply(lambda x: x.split(" ")[0]).astype(int)
df4['Engine'].head()

2     1199
11    1497
16    1498
25    1497
28    1199
Name: Engine, dtype: int32

In [31]:
df4['Brand'].value_counts()

Maruti           1197
Hyundai          1100
Honda             601
Toyota            409
Mercedes-Benz     318
Volkswagen        315
Name: Brand, dtype: int64

In [33]:
# Shuffle the dataset as we don't want it to be in a certain order while may cause some biasness in train test split
df5 = df4.sample(frac=1)

In [34]:
# we will drop the features 'Location' and 'Mileage' as they won't contribute much to our model
df5.drop(['Location', 'Mileage'], axis=1, inplace=True)

In [35]:
df5.head()

,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Engine,Power,Seats,Price,Brand,Age
5250,38000,Petrol,Manual,Second,1599,103.2 bhp,5.0,2.60,Hyundai,12
3343,68319,Diesel,Automatic,First,2982,168.5 bhp,7.0,25.12,Toyota,3
5622,35129,Petrol,Automatic,First,1197,83.1 bhp,5.0,7.00,Maruti,4
5344,53000,Petrol,Manual,First,1197,81.83 bhp,5.0,5.50,Hyundai,5
4677,49700,Diesel,Manual,Second,1396,88.76 bhp,5.0,4.18,Hyundai,8


### Encoding
<b> We will do encoding for for features fuel_type, Transmission, owner_type and brand, since we have to convert them to numeric form </b>


In [36]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()


df5['Fuel_Type'] =  enc.fit_transform(df5.Fuel_Type)
df5['Transmission'] = enc.fit_transform(df5.Transmission)
df5['Owner_Type'] = enc.fit_transform(df5.Owner_Type)
df5['Brand'] = enc.fit_transform(df5.Brand)

In [37]:
# we will perform some binning operation for column engine and kms drive for ease of model building

bins = [500, 900, 1100, 1300, 1700, 2000, 3000, 4000, 5000, 6000]
labels = [ 750, 1000, 1200, 1500, 1800, 2500, 3500, 4500, 5500]
df5['Engine_CC'] = pd.cut(df5['Engine'], bins=bins, labels=labels)

In [38]:
bins = [1000, 15000, 25000, 35000, 45000, 60000, 90000, 130000, 200000]
labels = [10000, 20000, 30000, 40000, 50000, 75000, 100000, 150000]
df5['Kms_Driven'] = pd.cut(df5['Kilometers_Driven'], bins=bins, labels=labels)

In [39]:
# Finall we will drop all the unnecessary columns and prepare our final dataset for train test split

df5 = df5.drop(['Kilometers_Driven', 'Engine'], axis=1)

In [40]:
df5.drop(['Power'], axis=1, inplace=True)

In [41]:
df5['Seats'] = df5['Seats'].astype(int)

In [42]:
df5.head()

,Fuel_Type,Transmission,Owner_Type,Seats,Price,Brand,Age,Engine_CC,Kms_Driven
5250,3,1,2,5,2.60,1,12,1500,40000
3343,1,0,0,7,25.12,4,3,2500,75000
5622,3,0,0,5,7.00,2,4,1200,40000
5344,3,1,0,5,5.50,1,5,1200,50000
4677,1,1,2,5,4.18,1,8,1500,50000


In [43]:
# I observed some NA values after binning, those could be extreme values or ouliers, we will drop to avoid complications
df5.dropna(inplace=True)

### Feature Selection

In [44]:
X = df5.drop(['Price'], axis=1)
y = df5['Price']

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [46]:
print(X_train.shape)
print(X_test.shape)

(3130, 8)
(783, 8)


### Standardization

<b> First we will build model without standardization. If it performs well then we could avoid it completely or else we can use standardized dataset later for better model accuracy.</b>

In [47]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()

# Xtrain_sd = X.fit_transform(X_train)
# Xtest_sd = X.transform(X_test)

### Model building

I am going with the conventinal Linear Regression as a base model and Random Forest Regressor as an ultimate one. You could try some other models as well if you wish. But since my aim is to deploy model, I will keep it as simple as possible.

In [48]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [49]:
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor()
reg.fit(X_train, y_train)
y_predict = reg.predict(X_test)

In [52]:
# Finally we will dump model and use it in our flask or any other web application file

import joblib
joblib.dump(reg,'price_model.ml')

['price_model.ml']